# How to use `indigopy`
Example code for how to use the `indigopy` package.

## Set up environment

In [1]:
# Import dependencies
import pandas as pd
from scipy.stats import spearmanr
from sklearn.metrics import r2_score, classification_report
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import plotly.graph_objects as go

# Import package functions
import sys
sys.path.append('c:/Users/User/github/INDIGOpy/') # modify if testing locally in different machine; remove once package is published
from indigopy.core import load_sample, featurize, classify

## Example: *E. coli*

In [2]:
# Load sample data
sample = load_sample('ecoli')

# Define input arguments
key             = sample['key']
profiles        = sample['profiles']
feature_names   = sample['feature_names']
train_ixns      = sample['train']['interactions']
train_scores    = sample['train']['scores']
test_ixns       = sample['test']['interactions']
test_scores     = sample['test']['scores']

# Determine ML features
train_data      = featurize(train_ixns, profiles, feature_names=feature_names, key=key, silent=True)
test_data       = featurize(test_ixns, profiles, feature_names=feature_names, key=key, silent=True)
X_train, X_test = train_data['feature_df'].to_numpy().transpose(), test_data['feature_df'].to_numpy().transpose()

# Determine class labels
thresh, classes = (-0.5, 2), ('S', 'N', 'A')
train_labels    = classify(train_scores, thresholds=thresh, classes=classes)
test_labels     = classify(test_scores, thresholds=thresh, classes=classes)

# Train and apply a regression-based model
reg_model = RandomForestRegressor()
reg_model.fit(X_train, train_scores)
reg_y = reg_model.predict(X_test)
r, p = spearmanr(test_scores, reg_y)
r2 = r2_score(test_scores, reg_y)
print('Regression results:')
print('\tSpearman R = {}'.format(round(r, 4)))
print('\tSpearman p = {:.3g}'.format(p))
print('\tR2 = {}'.format(round(r2, 4)))

# Train and apply a classification-based model
class_model = RandomForestClassifier()
class_model.fit(X_train, train_labels)
class_y = class_model.predict(X_test)
print('Classification results:')
print(classification_report(test_labels, class_y))

Regression results:
	Spearman R = 0.6823
	Spearman p = 2.82e-10
	R2 = 0.4148
Classification results:
              precision    recall  f1-score   support

           A       0.50      0.31      0.38        13
           N       0.68      0.93      0.79        42
           S       1.00      0.09      0.17        11

    accuracy                           0.67        66
   macro avg       0.73      0.44      0.45        66
weighted avg       0.70      0.67      0.60        66



## Example: *M. tuberculosis*

In [3]:
# Load sample data
sample = load_sample('mtb')

# Define input arguments
key             = sample['key']
profiles        = sample['profiles']
feature_names   = sample['feature_names']
train_ixns      = sample['train']['interactions']
train_scores    = sample['train']['scores']
test_ixns       = sample['test']['interactions']
test_scores     = sample['test']['scores']
clinical_ixns   = sample['clinical']['interactions']
clinical_scores = sample['clinical']['scores']

# Determine ML features
train_data      = featurize(train_ixns, profiles, feature_names=feature_names, key=key, silent=True)
test_data       = featurize(test_ixns, profiles, feature_names=feature_names, key=key, silent=True)
clinical_data   = featurize(clinical_ixns, profiles, feature_names=feature_names, key=key, silent=True)
X_train, X_test = train_data['feature_df'].to_numpy().transpose(), test_data['feature_df'].to_numpy().transpose()
X_clinical      = clinical_data['feature_df'].to_numpy().transpose()

# Determine class labels
thresh, classes = (0.9, 1.1), ('S', 'N', 'A')
train_labels    = classify(train_scores, thresholds=thresh, classes=classes)
test_labels     = classify(test_scores, thresholds=thresh, classes=classes)

# Train and apply a regression-based model
reg_model = RandomForestRegressor()
reg_model.fit(X_train, train_scores)
reg_y = reg_model.predict(X_test)
r, p = spearmanr(test_scores, reg_y)
r2 = r2_score(test_scores, reg_y)
print('Regression results:')
print('\tSpearman R = {}'.format(round(r, 4)))
print('\tSpearman p = {:.3g}'.format(p))
print('\tR2 = {}'.format(round(r2, 4)))

# Train and apply a classification-based model
class_model = RandomForestClassifier()
class_model.fit(X_train, train_labels)
class_y = class_model.predict(X_test)
print('Classification results:')
print(classification_report(test_labels, class_y))

# Apply model to clinical data
clinical_y = reg_model.predict(X_clinical)
r, p = spearmanr(clinical_scores, clinical_y)
print('Clinical results:')
print('\tSpearman R = {}'.format(round(-r, 4)))
print('\tSpearman p = {:.3g}'.format(p))

Regression results:
	Spearman R = 0.5883
	Spearman p = 0.000161
	R2 = 0.1507
Classification results:
              precision    recall  f1-score   support

           A       0.67      0.38      0.48        16
           N       0.00      0.00      0.00         1
           S       0.73      0.84      0.78        19

    accuracy                           0.61        36
   macro avg       0.46      0.41      0.42        36
weighted avg       0.68      0.61      0.63        36

Clinical results:
	Spearman R = 0.5607
	Spearman p = 5.74e-06


## Example: *S. aureus*

In [4]:
# Load sample data
sample = load_sample('saureus')

# Define input arguments
key             = sample['key']
profiles        = sample['profiles']
feature_names   = sample['feature_names']
train_ixns      = sample['train']['interactions']
train_scores    = sample['train']['scores']
test_ixns       = sample['test']['interactions']
test_scores     = sample['test']['scores']
strains         = sample['orthology']['strains']
orthology_map   = sample['orthology']['map']

# Determine ML features
train_data      = featurize(train_ixns, profiles, feature_names=feature_names, key=key, silent=True)
test_data       = featurize(test_ixns, profiles, feature_names=feature_names, key=key, silent=True)
X_train, X_test = train_data['feature_df'].to_numpy().transpose(), test_data['feature_df'].to_numpy().transpose()

# Determine class labels
thresh, classes = (-0.5, 2), ('S', 'N', 'A')
train_labels    = classify(train_scores, thresholds=thresh, classes=classes)
test_labels     = classify(test_scores, thresholds=thresh, classes=classes)

# Train and apply a regression-based model
reg_model = RandomForestRegressor()
reg_model.fit(X_train, train_scores)
reg_y = reg_model.predict(X_test)
r, p = spearmanr(test_scores, reg_y)
r2 = r2_score(test_scores, reg_y)
print('Regression results:')
print('\tSpearman R = {}'.format(round(r, 4)))
print('\tSpearman p = {:.3g}'.format(p))
print('\tR2 = {}'.format(round(r2, 4)))

# Orthology version
test_data_ortho = featurize(test_ixns, profiles, feature_names=feature_names, key=key, 
                            strains=strains, orthology_map=orthology_map, silent=True)
X_test_ortho    = test_data_ortho['feature_df'].to_numpy().transpose()
reg_y_ortho = reg_model.predict(X_test_ortho)
r, p = spearmanr(test_scores, reg_y_ortho)
r2 = r2_score(test_scores, reg_y_ortho)
print('Regression results:')
print('\tSpearman R = {}'.format(round(r, 4)))
print('\tSpearman p = {:.3g}'.format(p))
print('\tR2 = {}'.format(round(r2, 4)))

# Train and apply a classification-based model
class_model = RandomForestClassifier()
class_model.fit(X_train, train_labels)
class_y = class_model.predict(X_test)
print('Classification results:')
print(classification_report(test_labels, class_y))

Regression results:
	Spearman R = 0.4792
	Spearman p = 0.000868
	R2 = -0.3171
Regression results:
	Spearman R = 0.2354
	Spearman p = 0.119
	R2 = -0.9628
Classification results:
              precision    recall  f1-score   support

           A       0.33      0.50      0.40         2
           N       0.50      0.77      0.61        22
           S       0.62      0.24      0.34        21

    accuracy                           0.51        45
   macro avg       0.49      0.50      0.45        45
weighted avg       0.55      0.51      0.48        45



## Example: *A. baumannii*

In [20]:
# Load sample data
sample = load_sample('abaumannii')

# Define input arguments
key             = sample['key']
profiles        = sample['profiles']
feature_names   = sample['feature_names']
train_ixns      = sample['train']['interactions']
train_scores    = sample['train']['scores']
test_ixns       = sample['test']['interactions']
test_scores     = sample['test']['scores']
strains         = sample['orthology']['strains']
orthology_map   = sample['orthology']['map']

# Determine ML features
train_data      = featurize(train_ixns, profiles, feature_names=feature_names, key=key, silent=True)
test_data       = featurize(test_ixns, profiles, feature_names=feature_names, key=key, silent=True)
X_train, X_test = train_data['feature_df'].to_numpy().transpose(), test_data['feature_df'].to_numpy().transpose()

# Determine class labels
thresh, classes = (-0.5, 0), ('S', 'N', 'A')
train_labels    = classify(train_scores, thresholds=thresh, classes=classes)
test_labels     = classify(test_scores, thresholds=thresh, classes=classes)

# Train and apply a regression-based model
reg_model = RandomForestRegressor()
reg_model.fit(X_train, train_scores)
reg_y = reg_model.predict(X_test)
r, p = spearmanr(test_scores, reg_y)
r2 = r2_score(test_scores, reg_y)
print('Regression results:')
print('\tSpearman R = {}'.format(round(r, 4)))
print('\tSpearman p = {:.3g}'.format(p))
print('\tR2 = {}'.format(round(r2, 4)))

# Orthology version
test_data_ortho = featurize(test_ixns, profiles, feature_names=feature_names, key=key, 
                            strains=strains, orthology_map=orthology_map, silent=True)
X_test_ortho    = test_data_ortho['feature_df'].to_numpy().transpose()
reg_y_ortho = reg_model.predict(X_test_ortho)
r, p = spearmanr(test_scores, reg_y_ortho)
r2 = r2_score(test_scores, reg_y_ortho)
print('Regression results:')
print('\tSpearman R = {}'.format(round(r, 4)))
print('\tSpearman p = {:.3g}'.format(p))
print('\tR2 = {}'.format(round(r2, 4)))

# Train and apply a classification-based model
class_model = RandomForestClassifier()
class_model.fit(X_train, train_labels)
class_y = class_model.predict(X_test)
print('Classification results:')
print(classification_report(test_labels, class_y))

# Visualize results
df = pd.DataFrame({
    'x': test_labels, 
    'y': reg_y_ortho
}
)
fig = go.Figure()
fig.add_trace(go.Box(y=df.y[df.x=='A'], name='Antagonism', marker_color='red'))
fig.add_trace(go.Box(y=df.y[df.x=='N'], name='Neutral', marker_color='gray'))
fig.add_trace(go.Box(y=df.y[df.x=='S'], name='Synergy', marker_color='blue'))
fig.update_layout(
    autosize=False,
    width=500,
    height=500, 
    title='A. baumannii', 
    xaxis_title='True Class', 
    yaxis_title='Predicted Score', 
    showlegend=False, 
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

Regression results:
	Spearman R = 0.63
	Spearman p = 3.53e-06
	R2 = -0.6869
Regression results:
	Spearman R = 0.5294
	Spearman p = 0.000185
	R2 = -0.0752
Classification results:
              precision    recall  f1-score   support

           A       0.47      0.82      0.60        17
           N       0.17      0.09      0.12        11
           S       0.78      0.41      0.54        17

    accuracy                           0.49        45
   macro avg       0.47      0.44      0.42        45
weighted avg       0.51      0.49      0.46        45

